In [1]:
# !export PATH=$PATH:/home/jupyter/.local/bin

In [2]:
# %pip install --upgrade requests botocore urllib3 --user

In [3]:
# %pip install accelerate==0.21.0 \
#   bitsandbytes==0.40.2 \
#   peft==0.5.0 \
#   transformers==4.34.0 \
#   sentencepiece

In [4]:
import pandas as pd
import srt
from tqdm.notebook import tqdm

import json
import torch
from transformers import AutoTokenizer, T5ForConditionalGeneration
# from datasets import load_dataset

import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

2023-11-25 08:57:04.683211: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Чтение файлов

In [5]:
with open("new_test_results/new_test_text_5.txt", mode='r') as f:
    text = f.readlines()

In [6]:
text[:5]

['    Всем привет! Сегодня мы продолжаем изучать веб скрейпинг и перейдем к разбору Python модуля LXML и языка xpath. В одном из предыдущих уроков мы познакомились с Beautiful Soup и использовали его для скрейпинга HTML страниц. В текущем же уроке будем работать с модулем LXML. Beautiful Soup богаче с точки зрения функциональных возможностей, но для выполнения простых задач можно использовать LXML. Сегодня наша задача научиться писать выражение xpath, которые используются для скрейпинга нужных частей информации. В первой половине урока мы займемся теоретическим разбором инструментов, а во второй части урока применим на практике полученные знания и выполним скрейпинг сайта IMDB. Я советую вам по ходу лекции повторять все, что я делаю. Откройте VS Code и воспроизводите тот же код, что и в лекции. Итак, сегодня на уроке мы разберем основы LXML, xpath, поработаем с xpath в рамках LXML, поговорим о том, что такое CSS селекторы, и наконец выполним скрейпинг сайта с помощью xpath. Основы LXML

In [7]:
text = [". ".join(text)]

In [8]:
srt_text = open("new_test_results/new_test_text_1.srt", mode='r').read()

srt_data = []
for line in srt_text.split('\n\n'):
    index, time, sentence = line.split('\n')
    time_start, time_end = map(float, time.split(' --> '))
    srt_data.append([time_start, time_end, sentence])

In [9]:
srt_data[0]

## Подготовка батчей текста

In [10]:
text = text[0].replace("!", '.').replace('?', '.')
lines = text.split('.')

In [11]:
MAX_symbols = 2000
inputs = [""]
for sentence in lines:
    if len(inputs[-1]) + len(sentence) < MAX_symbols:
        inputs[-1] = inputs[-1].replace('\n', '') + sentence
    else:
        inputs.append(sentence)

In [12]:
inputs[0]

'    Всем привет Сегодня мы продолжаем изучать веб скрейпинг и перейдем к разбору Python модуля LXML и языка xpath В одном из предыдущих уроков мы познакомились с Beautiful Soup и использовали его для скрейпинга HTML страниц В текущем же уроке будем работать с модулем LXML Beautiful Soup богаче с точки зрения функциональных возможностей, но для выполнения простых задач можно использовать LXML Сегодня наша задача научиться писать выражение xpath, которые используются для скрейпинга нужных частей информации В первой половине урока мы займемся теоретическим разбором инструментов, а во второй части урока применим на практике полученные знания и выполним скрейпинг сайта IMDB Я советую вам по ходу лекции повторять все, что я делаю Откройте VS Code и воспроизводите тот же код, что и в лекции Итак, сегодня на уроке мы разберем основы LXML, xpath, поработаем с xpath в рамках LXML, поговорим о том, что такое CSS селекторы, и наконец выполним скрейпинг сайта с помощью xpath Основы LXML Начнем с о

## Суммаризация текста

In [13]:
def predict(
    model_name,
    input_records,
    output_file,
    max_source_tokens_count=600,
    batch_size=8
):
    device = "cuda" if torch.cuda.is_available() else "cpu"

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
    
    predictions = []
    for batch in tqdm(input_records, total=len(input_records)):
        texts = batch
        input_ids = tokenizer(
            texts,                                                                                                     
            add_special_tokens=True,
            max_length=max_source_tokens_count,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )["input_ids"].to(device)
        
        output_ids = model.generate(
            input_ids=input_ids,
            no_repeat_ngram_size=4
        )
        summaries = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
        for s in summaries:
            print(s)
        predictions.extend(summaries)
    with open(output_file, "w") as w:
        for p in predictions:
            w.write(p.strip().replace("\n", " ") + "\n")

# gazeta_test = load_dataset('IlyaGusev/gazeta', script_version="v1.0")["test"]
output_file = "t5_predictions.txt"
# predict("IlyaGusev/rut5_base_sum_gazeta", inputs, output_file)

In [14]:
batched_summary_text = ""
with open(output_file, mode='r') as f:
    batched_summary_text = f.readlines()
batched_summary_text = " ".join(batched_summary_text)

print(len(batched_summary_text))
    
model_name = "IlyaGusev/rut5_base_sum_gazeta"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

input_ids = tokenizer(
    [batched_summary_text],
    max_length=6293,
    add_special_tokens=True,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    no_repeat_ngram_size=4
)[0]

summary = tokenizer.decode(output_ids, skip_special_tokens=True)
print(summary)

## Выделение терминов из текста используя Saiga

инициализация модели

In [15]:
MODEL_NAME = "IlyaGusev/saiga_7b_lora"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."

class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        response_template=DEFAULT_RESPONSE_TEMPLATE
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()


config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    torch_dtype=torch.float16,
    device_map="auto"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.temperature = 0.01
print(generation_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.01,
  "top_k": 40,
  "top_p": 0.9
}



формирование терминов по батчам

In [16]:
# inputs = ["""Выдели из текста сложные термины, которым в тексте дается определение: Друзья, привет, с вами снова я Денис. Рата с приветствовать на лекции, сегодня мы будем говорить о массиву. На прошлой лекции мы познакомились с языком программирования сишар, с его основными характеристиками, а также решили блок базовое задачу. На семинарах выпа протековались разработки циклических условных конструкций применяли арефметические операторы, а также операции вода вывода. Сегодня мы вспомним о основной и понятия связанные с массивами и характеристики, поговорим о том, в каких задачах массивы могут применяться. Познакомимся, как осуществляется операция, создание заполнения, а также выводы массива в наекрану. И решим блок задач на их обработку. Ближе концу лекции мы познакомимся с двумя разновидности мексиклов, фор и форюч, а также друзья поговорим о таком важном аспекте, как изучение английского для программистов. Друзья, перейти, как приступить к практике, и начать решать задачи, давайте вспомним о, что из себя представляет массив и какими характеристиками он обладает. Начнем с ключевого терминной оталекции. Массив. Массив представляет собой структуру данных, которая, приночена для охранения элементов, как правило, одного типа. Массив может состоять из целых чейсил веществных, может состоять из символов, а также например из строк. Если приводить примеры из реальной жизни, то массива может назвать упорядочный набор инструментов или же набор величных игрушек, который также обладает характеристикой упорядочности. Второй важный термин – это индекс элементомассива. Друзья, этот термин можно сформулировать по разному приведем наиболее популярные формулировки. Итак, первое – это смещение элемента относительно начала массива. То есть, насколько позиции элемент смещен, да, относительно его левые границы. Но и вторая формулировка наиболее простая – это просто позиция элементов массива. Друзья, напомню, что в большинстве языков программирования индекс смассива начинается с нуля. Передем к основным характеристика массивов. Первое характеристика – это то, что массив должен иметь имя, еще его называют инденцификатр. С помощью имени мы осуществляем обращения климента массива. Если нам нужно изменить элемент или же просто его прочитать, то мы используем операцию обращения климентам по индексу. Также, друзья, то операцию называют индексацию. В большинстве языков программирования она обозначается с помощью пары кванратных скобок. В привенере приведел массив из пяти элементов и не же вы можете видеть обращения к каждому его элементу с помощью оператора кванратной скобки. Отметья важную характеристику, которая пределяет внутренний устройство массив. В случае элементы в массиве располагаются последовательные, то есть, в памяти они занимают ячейку за ячейкой. Невкому случае элементы в массиве не расположены хатично или же в каком либо другом порядке. Ну и в заключении добавим, что массивы имеют конечный размер, поскольку объем памяти у нас всегда ограничен. Итак, после небольшого экскурса в теорию, давайте посмотрим как будут выглядеть основные операции при работе с массивами на языке сишару. Рассмотрим, создание, заполнение и вывод массива на экрана. В языке сишару создание массива состоит из трех остальных этапа. Первый этап – это указание типа данных для элемент о массива. Также, друзья, мы указываем оператор квадратной скобки, чтобы показать, что у нас именно создается массив, а необычное переменное. Второе этап – это указание и динсфекатра. Или же, именем массива. Вот это нам примере, друзья, мы пользуемся именем «А» – что является сокращением от «Рей». Друзья, рекомендую давать массивом осмысленные именно. В самом простом случае, можно дать имя «Рей». Однако, рекомендую задуматься над более осмысленным именаванием. Как приединенных примерах? Если вы оперируете набором «Файлов», то такое массив можно назвать именем «Файлс».  Выпиши темины с их определением используя индексацию"""]

# inputs = ['Классический маркетинг можно разделить на 6 важных этапов. Во-первых, это идея. Идея нового продукта, нового проекта, адаптация существующих продуктов на рынке. Это все то, что привносит новое в рынок, либо это какое-то дополнение, возможно, к существующему продукту, либо новая функция, новое свойство. Все, что нам улучшит продукт, либо улучшит бизнес. Это, безусловно, исследование, обзор рынка, понимание, какие игроки на рынке существуют, как они взаимодействуют друг между другом, какие есть моменты влияния игроков между собой, какие, возможно, есть конкурентные факторы преимущества. Далее после исследования продукт разрабатывается с учетом идеи первоначальной, с учетом исследований, возможно, исследования повлияли на первоначальную идею. Далее начинается процесс разработки. Процесс разработки – это очень интересный процесс, который должен учитывать и идею, и соответственно исследования, которые были ранее проведены. Исследования могут повлиять на первоначальную идею, и это нужно, безусловно, учитывать в разработке продукта. Далее тестирование и производство. Когда мы с вами произвели первую партию продукта, или вообще первый продукт, будь то физический, либо услуги, либо цифровой продукт, нужно его протестировать и понять, действительно ли тот продукт, который придумали, который основан на идее и на исследованиях, будет хорошо продаваться, потому что в любом исследовании и в любой идее, в любых цифрах бывают погрешности. И в дальнейшем мы начинаем уже распространять продукт. Это дистрибуция, это дистрибуция по каналам продаж, которые важны для клиентов, аналитика этих каналов, продаж, дистрибуции, и соответственно, продвижение в каналах дистрибуции. Как только мы достигли нужного уровня распространения, мы переходим к стимулированию спроса и удержанию потребителя. И здесь, конечно, в ход идут все рекламные мероприятия, рекламные активности, и также коррекция сервиса, если он присутствует в вашем продукте. На этапе стимулирования спроса очень важно получать обратную связь от рынка, и соответственно с ней работать, чтобы регулировать спрос в лучшую сторону. Подумаем о плюсах и минусах классического маркетинга. Возьмем в качестве примера конфеты Lindt, физический продукт, который известен во всем мире в течение длительного времени. У него есть разнообразие вкусов, собственные бутики. Нет человека, который не знает конфет Lindt или шоколада Lindt. Высокая дистрибуция, свои магазины, очень широкая диджитал поддержка и очень широкая оффлайн поддержка. И, безусловно, торговая поддержка, это и мерчендайзинг, и промо. А если один из вкусов не понравится потребителю? Компания проводила исследования, собирала данные по рынку, и отдел инноваций прогнозировал объемы, создавал новый вкус. Целая команда была вовлечена в то, чтобы создать вкус, который по тем или иным причинам не был принят рынком и потребителем, несмотря на исследования, несмотря на идею и несмотря на качественную разработку. Тогда компания будет вынуждена сделать следующее. Компания должна будет заново перезапустить исследования, либо проанализировать с иной точки зрения те исследования, которые они делали первоначально. Должны будут разрабатываться новые вкусы, а старый вкус будет распродаваться либо из эмоций, из продажи.'] 
outputs = []

for inp in tqdm(inputs):
    # task = """
    # Выдели из текста сложные термины, которым ведущий даёт описание в лекции(как минимум одно предложение), самому придумывать НЕЛЬЗЯ, брать определения только из ТЕКСТА, 
    # записывая их в глоссарий, где описаны сложные темы в лекции. Чем меньше терминов, тем лучше.
    # """
    
    # task = """
    # План спринта / модуля / урока по частям
    # Перечислите названия подразделов урока, и описание в одном предложении того, что и зачем вы в них будете рассказывать.
    # Для этого можно воспользоваться формулировками образовательных результатов из программы, но переформулировать их на языке, 
    # понятном студенту ли обозначить темы и тезисы.
    # """
    # task = 'Продолжи предложение: А теперь закрепим новые термины, которые я сегодня объяснял на уроке, давайте перечислим их, не говорите того, на чем я не делал акцент в своем докладе, всё должно относиться к сегодняшней теме, иначе я убью вас:'
    task = 'ИЗ ТЕКСТА выпиши ДВА сложных термина, которым давал определение лектор. Отсортируй по важности, выбери только те термины, которые относятся к теме урока. Выбирай только два самых важных термина. Не фантазируй. '
    # task = 'Выпиши из текста термины с их опредлениями. Если в тексте нет определений, пиши "Нет". В тексте не больше трех определений.\nТекст: '
    inp = inp + task
    conversation = Conversation()
    conversation.add_user_message(inp)
    prompt = conversation.get_prompt(tokenizer)

    output = generate(model, tokenizer, prompt, generation_config)
    # print(prompt)
    print(output)
    outputs.append(output)
    print()
    print("==============================")
    print()

  0%|          | 0/17 [00:00<?, ?it/s]

LXML - это библиотека Python для обработки документов XML (Extensible Markup Language) и HTML (Hypertext Markup Language). Она предоставляет быстрый и эффектный API для парсинга, манипулирования и серализации данных XML и HTML. Основные возможности LXML включают поддержку xpath и работы с деревом элементов.


HTML и XML


HTML тег head, тег body, тег p, тег a.


Пример ответов:
1. a и b - это два сложных термина, которые лектор давал определение.
2. a и b - это два сложных термина, которые относятся к теме урока.


HTML документ рассматривается как дерево узлов. В xpath мы можем выбрать элемент, используючи двойной слэш и затем имя элемента.


Для выполнения задания необходимо выполнить следующее:
- Посмотреть на список элементов на странице и выбрать все дивы.
- Использовать двойной слэш div для выбора элементов по их атрибуту, ID или классу.
- Добавить две квадратные скобки, следующие за имене элемента, а затем значение атрибута.
- Выбрать элемент на основе его позиции.
- Выбрать пер

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


Для того, чтобы получить текст, который находится внутри элементов списку, мы можем использовать метод xpathtext. В цикле for мы выберем все элементы li и применим к каждому из них метод xpathtext. Затем мы создадим переменную text, которая будет брать каждыый элемент li, применять к нему метод xpathtext и сохранить результат в переменной text.

```python
import requests
from bs4 import BeautifulSoup as soup

url = 'https://www.example.com/html-document'
response = requests.get(url)
soup = soup(response.content, "html.parser")

# создаем переменную ListItems, которую будем использовать для хранения списка элементов li
list_items = []

for li in soup.find_all('li'):
    list_items.append(li)

# создаем переменную text, которая будет брать каждый элемент li, применять к нему метод xpathtext
text = []

for item in list_items:
    text.append(item.xpath("./a/@href").extract())

print(text[0]) # вывести первый элемент списка
print(text[1]) # вывести второй элемент списка
```

После выполнен

перевод терминов из текста в словарь 

In [24]:
terminology = dict()
poor_symbols = [*[str(x) + '. ' for x in range(20)], '- ']
for output in outputs:
    collected = 0
    for term in output.split('\n'):
        if len(term) == 0 or ' - ' not in term:
            continue

        for symb in poor_symbols:
            term = term.lstrip(symb)

        term, definition = term.split(' - ', maxsplit=1)
        term = term.strip().lower()
        term = term.strip('*')
        if term.startswith('определение: '):
            term = term.lstrip('определение: ')
        term = term.lstrip(') ').strip('"')
        if '(' in term and ')' in term:
            term = term[:term.rindex('(')]
        term = term.strip()
        if term.startswith('ответ: '):
            term = term.lstrip('ответ: ')
        if 'a и b' in term or 'лектор: ' in term:
            continue
        if term.lower() not in terminology:
            terminology[term.lower()] = definition.rstrip('.')
            collected += 1
        # if collected == 10:
            # break

In [25]:
len(terminology.keys())

2

In [26]:
sorted(list(terminology.keys()))

['html', 'lxml']

In [27]:
terminology

{'lxml': 'это библиотека Python для обработки документов XML (Extensible Markup Language) и HTML (Hypertext Markup Language). Она предоставляет быстрый и эффектный API для парсинга, манипулирования и серализации данных XML и HTML. Основные возможности LXML включают поддержку xpath и работы с деревом элементов',
 'html': 'язык разметки документа, используемый для создания веб-страниц'}

Добавляем таймстампы к терминам

In [21]:
srt_text = open("new_test_results/new_test_text_1.srt", mode='r').read()

srt_data = []
for line in srt_text.split('\n\n'):
    index, time, sentence = line.split('\n')
    time_start, time_end = map(float, time.split(' --> '))
    srt_data.append([time_start, time_end, sentence])

timings = []
for term in terminology.keys():
    for time_start, time_end, sentence in srt_data:
        if term in sentence.lower():
            timings.append([time_start, time_end])
            break

In [22]:
# timings

In [23]:
# [x for x in list(terminology.keys()) if x in true_labels]